In [1]:
import numpy as np
import pandas as pd
import psutil as ps
import os
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from Utility import draw_heatmap, draw_bitmap, draw_importance_forest, cross_val_RF, cross_val_Lin, cross_val_NN
import time

nb_fold = 10
nb_feature = 5
df = pd.read_csv('./data/train.csv').values
train_df, test_df = train_test_split(df, test_size=0.3)
X_train = train_df[:, 2:2+nb_feature]
y_train = train_df[:, -1]
X_test = test_df[:, 2:2+nb_feature]
y_test = test_df[:, -1]
print X_train.shape, y_train.shape
print X_train[:3], y_train[:3]
print X_test.shape, y_test.shape
print X_test[:3], y_test[:3]

Using TensorFlow backend.


(335, 5) (335,)
[[27.51 11.85 24.377 11.978 19]
 [62.5 17.33 69.175 11.247 19]
 [52.79561999999999 15.04667 51.446000000000005 17.965999999999998 19]] [-87.96666667 -124.9 -86.3]
(144, 5) (144,)
[[27.51097 12.31091 24.377 11.978 19]
 [80.47 22.65 82.947 18.112000000000002 19]
 [31.65 4.1 24.377 11.978 19]] [-81.06666667 -79.5 -104.4]


In [4]:
y_pred = searcher.predict(X_test)
#print y_pred[:10]
#print y_test[:10]
print "--- Model--- %s features" %(nb_feature)
print "[MSE]: %.3f" % mean_squared_error(y_test, y_pred)
print "[RMSE]: %.3f" % (mean_squared_error(y_test, y_pred))**(0.5)
print '[R2]: %.3f' % r2_score(y_test, y_pred)

--- Model--- 5 features
[MSE]: 28.884
[RMSE]: 5.374
[R2]: 0.836


In [46]:
#Decision tree
model = DecisionTreeRegressor()
param_grid = {'max_depth': [1, 10, 20, 30, 40, 50]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.758684702701
{'max_depth': 30}


In [118]:
#Random forest
model = RandomForestRegressor()
param_grid = {
    'n_estimators': [1, 50, 100],
    'max_depth': [10, 30, 50],
    'max_features': ['sqrt', 'auto'] 
}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.847706700719
{'max_features': 'auto', 'n_estimators': 50, 'max_depth': 50}


In [116]:
#Gradient Boosting
model = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [1, 50, 100],
    'max_depth': [1, 2, 3],
    'learning_rate' : [0.1, 0.01] 
}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.842648857024
{'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}


In [123]:
# K-NN
model = KNeighborsRegressor()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.809960920268
{'n_neighbors': 3}


In [18]:
#Linear regression
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [2]:
#NN
begin_time = time.time()
def create_model():
    model = Sequential()    
    model.add(Dense(512, activation='relu', input_dim = nb_feature)) 
    model.add(Dense(512, activation='relu'))    
    model.add(Dense(512, activation='relu'))     
    model.add(Dense(512, activation='relu'))   
    model.add(Dense(1))     
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
model = KerasRegressor(build_fn=create_model, epochs=1000)
param_grid = {'batch_size' : [16, 32]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)
searcher.fit(X_train, y_train)
print searcher.best_score_
print searcher.best_params_
print time.time() - begin_time

Epoch 1/1000
Epoch 1/1000
301/301 [==============================] - 1s - loss: 2911.8172     
Epoch 2/1000
160/301 [==============>...............] - ETA: 0s - loss: 778.5338
Epoch 2/1000
301/301 [==============================] - 1s - loss: 743.2866     
Epoch 3/1000
288/301 [===========================>..] - ETA: 0s - loss: 453.9914
Epoch 3/1000
301/301 [==============================] - 1s - loss: 459.3306     
Epoch 4/1000
301/301 [==============================] - 1s - loss: 240.3165     
Epoch 5/1000
 64/301 [=====>........................] - ETA: 1s - loss: 174.4559
Epoch 4/1000
301/301 [==============================] - 1s - loss: 211.9657     
Epoch 5/1000
301/301 [==============================] - 1s - loss: 141.3272     
Epoch 7/1000
301/301 [==============================] - 1s - loss: 144.9908     
Epoch 6/1000
301/301 [==============================] - 1s - loss: 127.2572     
Epoch 7/1000
301/301 [==============================] - 2s - loss: 123.8742     
Epoch 8/1000
3

301/301 [==============================] - 1s - loss: 56.1869     
Epoch 48/1000
 16/301 [>.............................] - ETA: 0s - loss: 65.8358
Epoch 49/1000
301/301 [==============================] - 1s - loss: 55.4688     
Epoch 50/1000
301/301 [==============================] - 1s - loss: 61.7676     
Epoch 49/1000
301/301 [==============================] - 1s - loss: 46.8457     
Epoch 50/1000
301/301 [==============================] - 1s - loss: 55.0571     
Epoch 52/1000
301/301 [==============================] - 1s - loss: 51.8851     
Epoch 51/1000
301/301 [==============================] - 1s - loss: 65.7720     
Epoch 53/1000
301/301 [==============================] - 1s - loss: 49.1899     
Epoch 52/1000
301/301 [==============================] - 1s - loss: 54.5999     
Epoch 54/1000
 64/301 [=====>........................] - ETA: 0s - loss: 59.4129
Epoch 53/1000
240/301 [======================>.......] - ETA: 0s - loss: 55.6942
Epoch 55/1000
301/301 [===================

KeyboardInterrupt: 